In [1]:
#a. Prepare Data

#Load the data from the ratings.csv and movies.csv files and combine them on movieId. The resultant data set should contain all of the user ratings and include movie titles. The schema should look something like this.

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder.appName('Week11').getOrCreate()

In [4]:
movies = spark.read.load(r'/FileStore/tables/week11/movies.csv', format='csv', inferSchema=True, header=True)
ratings = spark.read.load(r'/FileStore/tables/week11/ratings.csv', format='csv', inferSchema=True, header=True)

In [5]:
df = movies.join(ratings, 'movieId', 'inner')

In [6]:
df.show()

+-------+--------------------+--------------------+------+------+---------+
movieId| title| genres|userId|rating|timestamp|
+-------+--------------------+--------------------+------+------+---------+
 1| Toy Story (1995)|Adventure|Animati...| 1| 4.0|964982703|
 3|Grumpier Old Men ...| Comedy|Romance| 1| 4.0|964981247|
 6| Heat (1995)|Action|Crime|Thri...| 1| 4.0|964982224|
 47|Seven (a.k.a. Se7...| Mystery|Thriller| 1| 5.0|964983815|
 50|Usual Suspects, T...|Crime|Mystery|Thr...| 1| 5.0|964982931|
 70|From Dusk Till Da...|Action|Comedy|Hor...| 1| 3.0|964982400|
 101|Bottle Rocket (1996)|Adventure|Comedy|...| 1| 5.0|964980868|
 110| Braveheart (1995)| Action|Drama|War| 1| 4.0|964982176|
 151| Rob Roy (1995)|Action|Drama|Roma...| 1| 5.0|964984041|
 157|Canadian Bacon (1...| Comedy|War| 1| 5.0|964984100|
 163| Desperado (1995)|Action|Romance|We...| 1| 5.0|964983650|
 216|Billy Madison (1995)| Comedy| 1| 5.0|964981208|
 223| Clerks (1994)| Comedy| 1| 3.0|964980985|
 231|Dumb & Dumber (Du...| Adventure|Comedy| 1| 5.0|964981179|
 235| Ed Wood (1994)| Comedy|Drama| 1| 4.0|964980908|
 260|Star Wars: Episod...|Action|Adventure|...| 1| 5.0|964981680|
 296| Pulp Fiction (1994)|Comedy|Crime|Dram...| 1| 3.0|964982967|
 316| Stargate (1994)|Action|Adventure|...| 1| 3.0|964982310|
 333| Tommy Boy (1995)| Comedy| 1| 5.0|964981179|
 349|Clear and Present...|Action|Crime|Dram...| 1| 4.0|964982563|
+-------+--------------------+--------------------+------+------+---------+
only showing top 20 rows

In [7]:
#b. Train Recommender

#Using the data you prepared in the last step, create a movie recommendation model using collaborative filtering. Spark’s collaborative filtering documentation provides a template for building and testing this model.

#Before you train the recommendation model, split the data into a training dataset and a testing dataset using the randomSplit dataframe method. Use 80% of your data for training and 20% for testing.

#After fitting your model using the training dataset, calculate the predictions on the test dataset and use the RegressionEvaluator to calculate the root-mean-square error of the model.

#As a reminder, Spark’s collaborative filtering documentation will be helpful in completing this task.

In [8]:
(training, test) = df.randomSplit([0.8, 0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [10]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(str(rmse))

1.0590834276801555

In [11]:
#c. Generate top 10 movie recommendations

#Using the recommendation model, generate the top ten recommendations for each user. Using the show method, print the recommendations for the user IDs, 127, 151, and 300. You should not truncate the results and so should call the show method like this recommendations_127.show(truncate=False).

In [12]:
user_ids = [127,151,300]
users = ratings.select(als.getUserCol()).distinct().where(col('userId').isin(user_ids))
userRecs = model.recommendForUserSubset(users, 10)

In [13]:
userRecs.show(truncate=False)

+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
userId|recommendations |
+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
300 |[[1241, 6.6229596], [3030, 6.497546], [4799, 6.442905], [190, 6.391551], [129354, 6.3647556], [70946, 6.3101273], [1194, 6.27455], [7387, 6.2273865], [3435, 6.186246], [3925, 6.051436]] |
127 |[[133419, 10.593578], [1694, 8.787968], [3404, 8.30903], [3688, 8.28609], [2469, 8.004135], [3836, 7.881112], [5478, 7.7974257], [1017, 7.6974344], [2398, 7.685412], [1250, 7.6757746]] |
151 |[[106100, 6.7422233], [3784, 6.65591], [7169, 6.648698], [96821, 6.3426213], [103228, 6.278337], [3053, 6.208957], [40629, 6.19634], [71282, 6.13091], [916, 6.1063533], [2401, 6.087534]]|
+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [14]:
for user_row in userRecs.collect():
    recommendations = []
    
    for rec in user_row['recommendations']:
        name = movies.select('title').where(movies.movieId==rec[0])
        recommendations.append({'name':name.collect()[0][0]})

    print(f'\nRecommendations for user {user_row[0]}\n')
    [print(f"{row['name']}") for row in recommendations]


Recommendations for user 300

Dead Alive (Braindead) (1992)
Yojimbo (1961)
It's a Mad, Mad, Mad, Mad World (1963)
Safe (1995)
Focus (2015)
Troll 2 (1990)
Cheech and Chong's Up in Smoke (1978)
Dawn of the Dead (1978)
Double Indemnity (1944)
Stranger Than Paradise (1984)

Recommendations for user 127

Pitch Perfect 2 (2015)
Apostle, The (1997)
Titanic (1953)
Porky's (1982)
Peggy Sue Got Married (1986)
Kelly's Heroes (1970)
Eight Legged Freaks (2002)
Swiss Family Robinson (1960)
Miracle on 34th Street (1947)
Bridge on the River Kwai, The (1957)

Recommendations for user 151

Dallas Buyers Club (2013)
Kid, The (2000)
Chasing Liberty (2004)
Perks of Being a Wallflower, The (2012)
Pacific Rim (2013)
Messenger: The Story of Joan of Arc, The (1999)
Pride & Prejudice (2005)
Food, Inc. (2008)
Roman Holiday (1953)
Pale Rider (1985)